# 6.2.1 特徵匹配挑戰 - 中級練習

本練習專注於特徵檢測、描述子提取和特徵匹配的實際應用。通過解決實際問題來深入理解特徵匹配的原理和技巧。

## 練習目標
- 掌握多種特徵檢測器的使用和比較
- 實現魯棒的特徵匹配算法
- 學會處理光照、角度變化等挑戰
- 應用Homography進行圖像配準
- 評估特徵匹配的效果和精度

## 難度等級: ⭐⭐⭐ (中級)

## 環境設置與導入

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
from typing import List, Tuple, Optional

# 添加utils路徑
sys.path.append('../../utils')
from image_utils import load_image, resize_image
from visualization import display_image, display_multiple_images
from performance import time_function, benchmark_function

# 設置matplotlib
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 環境設置完成")
print(f"OpenCV版本: {cv2.__version__}")

## 挑戰1: 多特徵檢測器比較分析

### 任務描述
實現一個特徵檢測器比較系統，能夠同時使用SIFT、ORB、BRISK等多種檢測器，並進行定量分析。

In [ ]:
class FeatureDetectorComparator:
    """特徵檢測器比較類"""
    
    def __init__(self):
        """初始化各種特徵檢測器"""
        self.detectors = {}
        self._initialize_detectors()
    
    def _initialize_detectors(self):
        """初始化所有可用的特徵檢測器"""
        try:
            # SIFT檢測器
            self.detectors['SIFT'] = cv2.SIFT_create()
            print("✅ SIFT檢測器初始化成功")
        except Exception as e:
            print(f"❌ SIFT初始化失敗: {e}")
        
        try:
            # ORB檢測器
            self.detectors['ORB'] = cv2.ORB_create(nfeatures=1000)
            print("✅ ORB檢測器初始化成功")
        except Exception as e:
            print(f"❌ ORB初始化失敗: {e}")
        
        try:
            # BRISK檢測器
            self.detectors['BRISK'] = cv2.BRISK_create()
            print("✅ BRISK檢測器初始化成功")
        except Exception as e:
            print(f"❌ BRISK初始化失敗: {e}")
        
        try:
            # AKAZE檢測器
            self.detectors['AKAZE'] = cv2.AKAZE_create()
            print("✅ AKAZE檢測器初始化成功")
        except Exception as e:
            print(f"❌ AKAZE初始化失敗: {e}")
        
        print(f"\n🔧 總共初始化了 {len(self.detectors)} 個檢測器")
    
    def detect_and_compute(self, image, detector_name):
        """
        使用指定檢測器進行特徵檢測和描述子計算
        
        Args:
            image: 輸入圖像
            detector_name: 檢測器名稱
        
        Returns:
            tuple: (關鍵點, 描述子, 處理時間)
        """
        if detector_name not in self.detectors:
            return None, None, 0
        
        detector = self.detectors[detector_name]
        
        # 轉換為灰階（如果需要）
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        # 檢測並計算描述子
        start_time = time.time()
        keypoints, descriptors = detector.detectAndCompute(gray, None)
        process_time = (time.time() - start_time) * 1000
        
        return keypoints, descriptors, process_time
    
    def compare_detectors(self, image_path):
        """
        比較所有檢測器在同一圖像上的表現
        
        Args:
            image_path: 圖像路徑
        """
        # 載入圖像
        image = load_image(image_path)
        if image is None:
            print(f"無法載入圖像: {image_path}")
            return
        
        # 調整圖像大小
        image_resized = resize_image(image, max_width=640)
        
        results = []
        titles = []
        comparison_data = []
        
        print(f"\n📊 特徵檢測器比較分析: {os.path.basename(image_path)}")
        print("-" * 60)
        
        for detector_name in self.detectors.keys():
            # 檢測特徵
            keypoints, descriptors, process_time = self.detect_and_compute(
                image_resized, detector_name
            )
            
            if keypoints is not None:
                # 繪製關鍵點
                img_with_kp = cv2.drawKeypoints(
                    image_resized, keypoints, None, 
                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
                )
                
                results.append(img_with_kp)
                titles.append(f"{detector_name}\n{len(keypoints)} kpts, {process_time:.1f}ms")
                
                # 收集比較數據
                desc_size = descriptors.shape[1] if descriptors is not None else 0
                comparison_data.append({
                    'name': detector_name,
                    'keypoints': len(keypoints),
                    'time': process_time,
                    'descriptor_size': desc_size
                })
                
                print(f"{detector_name:6}: {len(keypoints):4d} 特徵點, "
                      f"{process_time:6.1f}ms, 描述子維度: {desc_size}")
        
        # 顯示比較結果
        if results:
            all_images = [image_resized] + results
            all_titles = ["原始圖像"] + titles
            
            display_multiple_images(all_images, all_titles, figsize=(15, 10))
        
        # 輸出詳細統計
        self._print_detailed_statistics(comparison_data)
        
        return comparison_data
    
    def _print_detailed_statistics(self, data):
        """輸出詳細統計信息"""
        if not data:
            return
        
        print("\n📈 詳細統計分析:")
        print("-" * 60)
        
        # 按特徵點數量排序
        sorted_by_features = sorted(data, key=lambda x: x['keypoints'], reverse=True)
        print("特徵點數量排名:")
        for i, item in enumerate(sorted_by_features, 1):
            print(f"  {i}. {item['name']:6}: {item['keypoints']:4d} 特徵點")
        
        # 按速度排序
        sorted_by_speed = sorted(data, key=lambda x: x['time'])
        print("\n處理速度排名:")
        for i, item in enumerate(sorted_by_speed, 1):
            print(f"  {i}. {item['name']:6}: {item['time']:6.1f}ms")
        
        # 推薦使用場景
        print("\n💡 使用場景推薦:")
        fastest = sorted_by_speed[0]['name']
        most_features = sorted_by_features[0]['name']
        
        print(f"  • 實時應用: {fastest} (最快)")
        print(f"  • 高精度匹配: {most_features} (特徵點最多)")
        print("  • 移動設備: ORB (平衡速度和效果)")
        print("  • 學術研究: SIFT (經典算法)")

# 初始化比較器
comparator = FeatureDetectorComparator()

### 🎯 練習任務 1.1: 執行特徵檢測器比較

**任務**: 使用不同的測試圖像來比較各種特徵檢測器的效能。

In [ ]:
# 測試不同類型的圖像
test_images = [
    "../../assets/images/basic/faces01.jpg",  # 人臉圖像
    "../../assets/images/basic/faces.png",    # 複雜場景
    "../../assets/images/basic/face03.jpg"    # 單一物體
]

# 依次測試每個圖像
for image_path in test_images:
    if os.path.exists(image_path):
        print(f"\n🔍 測試圖像: {os.path.basename(image_path)}")
        comparison_results = comparator.compare_detectors(image_path)
    else:
        print(f"⚠️ 圖像不存在: {image_path}")

## 挑戰2: 魯棒特徵匹配系統

### 任務描述
實現一個能處理光照變化、尺度變換、旋轉等挑戰的魯棒特徵匹配系統。

In [ ]:
class RobustFeatureMatcher:
    """魯棒特徵匹配器"""
    
    def __init__(self, detector_name='SIFT', matcher_type='FLANN'):
        """
        初始化魯棒特徵匹配器
        
        Args:
            detector_name: 特徵檢測器名稱
            matcher_type: 匹配器類型 ('FLANN' or 'BF')
        """
        self.detector_name = detector_name
        self.matcher_type = matcher_type
        self.detector = self._create_detector(detector_name)
        self.matcher = self._create_matcher(matcher_type, detector_name)
        
    def _create_detector(self, name):
        """創建特徵檢測器"""
        detectors = {
            'SIFT': lambda: cv2.SIFT_create(),
            'ORB': lambda: cv2.ORB_create(nfeatures=1000),
            'BRISK': lambda: cv2.BRISK_create(),
            'AKAZE': lambda: cv2.AKAZE_create()
        }
        
        if name in detectors:
            return detectors[name]()
        else:
            print(f"未知檢測器: {name}, 使用SIFT")
            return cv2.SIFT_create()
    
    def _create_matcher(self, matcher_type, detector_name):
        """創建特徵匹配器"""
        if matcher_type == 'FLANN':
            if detector_name in ['SIFT', 'SURF']:
                # 基於樹的索引
                FLANN_INDEX_KDTREE = 1
                index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
            else:
                # LSH索引用於二進制描述子
                FLANN_INDEX_LSH = 6
                index_params = dict(algorithm=FLANN_INDEX_LSH,
                                  table_number=6,
                                  key_size=12,
                                  multi_probe_level=1)
            
            search_params = dict(checks=50)
            return cv2.FlannBasedMatcher(index_params, search_params)
        
        else:  # BruteForce
            if detector_name in ['ORB', 'BRISK', 'AKAZE']:
                return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            else:
                return cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    
    def match_features(self, img1, img2, ratio_threshold=0.75, min_matches=10):
        """
        執行特徵匹配
        
        Args:
            img1, img2: 輸入圖像
            ratio_threshold: Lowe's ratio test閾值
            min_matches: 最少匹配點數量
        
        Returns:
            dict: 匹配結果
        """
        # 檢測特徵點和描述子
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) if len(img1.shape) == 3 else img1
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) if len(img2.shape) == 3 else img2
        
        kp1, desc1 = self.detector.detectAndCompute(gray1, None)
        kp2, desc2 = self.detector.detectAndCompute(gray2, None)
        
        if desc1 is None or desc2 is None or len(desc1) < min_matches or len(desc2) < min_matches:
            return {
                'status': 'failed',
                'reason': 'insufficient_features',
                'keypoints1': kp1,
                'keypoints2': kp2
            }
        
        # 特徵匹配
        if self.matcher_type == 'FLANN' or not hasattr(self.matcher, 'match'):
            # 使用KNN匹配
            matches = self.matcher.knnMatch(desc1, desc2, k=2)
            
            # Lowe's ratio test
            good_matches = []
            for match_pair in matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_threshold * n.distance:
                        good_matches.append(m)
        else:
            # 直接匹配
            matches = self.matcher.match(desc1, desc2)
            good_matches = sorted(matches, key=lambda x: x.distance)
        
        if len(good_matches) < min_matches:
            return {
                'status': 'failed',
                'reason': 'insufficient_matches',
                'matches_found': len(good_matches),
                'keypoints1': kp1,
                'keypoints2': kp2
            }
        
        # 提取匹配點座標
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        
        # 計算Homography
        homography = None
        inliers_mask = None
        
        if len(good_matches) >= 4:
            homography, inliers_mask = cv2.findHomography(
                src_pts, dst_pts, cv2.RANSAC, 5.0
            )
        
        inliers_count = np.sum(inliers_mask) if inliers_mask is not None else 0
        
        return {
            'status': 'success',
            'keypoints1': kp1,
            'keypoints2': kp2,
            'matches': good_matches,
            'homography': homography,
            'inliers_mask': inliers_mask,
            'inliers_count': inliers_count,
            'total_matches': len(good_matches),
            'match_ratio': inliers_count / len(good_matches) if len(good_matches) > 0 else 0
        }
    
    def visualize_matches(self, img1, img2, match_result, max_matches=50):
        """
        視覺化匹配結果
        
        Args:
            img1, img2: 輸入圖像
            match_result: 匹配結果
            max_matches: 最大顯示匹配數
        
        Returns:
            np.array: 匹配視覺化圖像
        """
        if match_result['status'] != 'success':
            # 只顯示特徵點
            img1_kp = cv2.drawKeypoints(img1, match_result['keypoints1'], None)
            img2_kp = cv2.drawKeypoints(img2, match_result['keypoints2'], None)
            return np.hstack([img1_kp, img2_kp])
        
        # 繪製匹配結果
        matches_to_draw = match_result['matches'][:max_matches]
        
        if match_result['inliers_mask'] is not None:
            # 分別繪製內點和外點
            mask = match_result['inliers_mask'].ravel()[:len(matches_to_draw)]
            
            # 繪製所有匹配（外點為紅色）
            img_matches = cv2.drawMatches(
                img1, match_result['keypoints1'],
                img2, match_result['keypoints2'],
                matches_to_draw, None,
                matchColor=(0, 0, 255),  # 紅色
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )
            
            # 覆蓋繪製內點（綠色）
            inlier_matches = [m for i, m in enumerate(matches_to_draw) if i < len(mask) and mask[i]]
            if inlier_matches:
                img_matches = cv2.drawMatches(
                    img1, match_result['keypoints1'],
                    img2, match_result['keypoints2'],
                    inlier_matches, img_matches,
                    matchColor=(0, 255, 0),  # 綠色
                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS | cv2.DrawMatchesFlags_DRAW_OVER_OUTIMG
                )
        else:
            # 簡單繪製所有匹配
            img_matches = cv2.drawMatches(
                img1, match_result['keypoints1'],
                img2, match_result['keypoints2'],
                matches_to_draw, None,
                matchColor=(0, 255, 0),
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )
        
        return img_matches

# 創建魯棒匹配器
robust_matcher = RobustFeatureMatcher(detector_name='SIFT', matcher_type='FLANN')
print("✅ 魯棒特徵匹配器初始化完成")

### 🎯 練習任務 2.1: 測試魯棒特徵匹配

**任務**: 使用同一物體的不同視角圖像來測試特徵匹配的魯棒性。

In [ ]:
def test_robust_matching(img1_path, img2_path, detector_name='SIFT'):
    """
    測試魯棒特徵匹配
    
    Args:
        img1_path, img2_path: 圖像路徑
        detector_name: 檢測器名稱
    """
    # 載入圖像
    img1 = load_image(img1_path)
    img2 = load_image(img2_path)
    
    if img1 is None or img2 is None:
        print("無法載入圖像")
        return
    
    # 調整圖像大小
    img1 = resize_image(img1, max_width=500)
    img2 = resize_image(img2, max_width=500)
    
    # 創建匹配器
    matcher = RobustFeatureMatcher(detector_name=detector_name)
    
    # 執行匹配
    print(f"\n🔍 使用 {detector_name} 進行特徵匹配...")
    start_time = time.time()
    result = matcher.match_features(img1, img2)
    match_time = (time.time() - start_time) * 1000
    
    # 輸出結果
    print(f"匹配狀態: {result['status']}")
    print(f"處理時間: {match_time:.1f}ms")
    
    if result['status'] == 'success':
        print(f"總匹配點: {result['total_matches']}")
        print(f"內點數量: {result['inliers_count']}")
        print(f"匹配精度: {result['match_ratio']:.2%}")
        
        # 視覺化匹配結果
        img_matches = matcher.visualize_matches(img1, img2, result)
        
        title = (f"{detector_name} 匹配結果\n"
                f"匹配點: {result['total_matches']}, "
                f"內點: {result['inliers_count']}, "
                f"精度: {result['match_ratio']:.1%}")
        
        display_image(img_matches, title, figsize=(15, 8))
        
        return result
    else:
        print(f"匹配失敗: {result.get('reason', '未知原因')}")
        if 'matches_found' in result:
            print(f"找到的匹配點: {result['matches_found']}")
        
        return None

# 測試圖像匹配（使用相同場景的不同圖像）
test_pairs = [
    ("../../assets/images/basic/face03.jpg", "../../assets/images/basic/faces01.jpg"),
    ("../../assets/images/basic/faces.png", "../../assets/images/basic/faces01.jpg")
]

for img1_path, img2_path in test_pairs:
    if os.path.exists(img1_path) and os.path.exists(img2_path):
        print(f"\n📷 測試配對: {os.path.basename(img1_path)} <-> {os.path.basename(img2_path)}")
        test_robust_matching(img1_path, img2_path, 'SIFT')
    else:
        print(f"⚠️ 測試圖像不存在")

## 挑戰3: 圖像配準與拼接

### 任務描述
基於特徵匹配實現圖像配準和簡單的圖像拼接功能。

In [ ]:
class ImageRegistration:
    """圖像配準和拼接類"""
    
    def __init__(self, matcher=None):
        """初始化圖像配準器"""
        self.matcher = matcher or RobustFeatureMatcher()
    
    def register_images(self, img1, img2):
        """
        配準兩張圖像
        
        Args:
            img1: 參考圖像
            img2: 待配準圖像
        
        Returns:
            dict: 配準結果
        """
        # 執行特徵匹配
        match_result = self.matcher.match_features(img1, img2)
        
        if match_result['status'] != 'success':
            return {
                'status': 'failed',
                'reason': '特徵匹配失敗',
                'match_result': match_result
            }
        
        if match_result['homography'] is None:
            return {
                'status': 'failed',
                'reason': 'Homography計算失敗',
                'match_result': match_result
            }
        
        # 應用Homography變換
        h, w = img1.shape[:2]
        registered_img = cv2.warpPerspective(
            img2, match_result['homography'], (w, h)
        )
        
        # 計算配準誤差（使用RMSE）
        registration_error = self._calculate_registration_error(
            img1, registered_img
        )
        
        return {
            'status': 'success',
            'registered_image': registered_img,
            'homography': match_result['homography'],
            'registration_error': registration_error,
            'match_result': match_result
        }
    
    def _calculate_registration_error(self, img1, img2):
        """
        計算配準誤差
        
        Args:
            img1, img2: 輸入圖像
        
        Returns:
            float: RMSE誤差
        """
        # 轉換為灰階
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) if len(img1.shape) == 3 else img1
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) if len(img2.shape) == 3 else img2
        
        # 計算RMSE
        diff = (gray1.astype(np.float32) - gray2.astype(np.float32)) ** 2
        rmse = np.sqrt(np.mean(diff))
        
        return rmse
    
    def create_panorama(self, images):
        """
        創建全景圖像（簡化版）
        
        Args:
            images: 圖像列表
        
        Returns:
            np.array: 全景圖像
        """
        if len(images) < 2:
            return images[0] if images else None
        
        # 使用第一張圖像作為基準
        panorama = images[0].copy()
        
        print(f"📸 開始創建全景圖，共 {len(images)} 張圖像")
        
        for i in range(1, len(images)):
            print(f"正在處理第 {i+1} 張圖像...")
            
            # 配準當前圖像到全景圖
            result = self.register_images(panorama, images[i])
            
            if result['status'] == 'success':
                # 簡單的圖像融合（平均值）
                registered = result['registered_image']
                
                # 創建遮罩找到重疊區域
                mask = cv2.cvtColor(registered, cv2.COLOR_BGR2GRAY) > 0
                
                # 在重疊區域進行融合
                panorama_float = panorama.astype(np.float32)
                registered_float = registered.astype(np.float32)
                
                # 簡單平均融合
                overlap_mask = mask[:, :, np.newaxis]
                panorama_float[overlap_mask] = (
                    panorama_float[overlap_mask] + registered_float[overlap_mask]
                ) / 2.0
                
                panorama = panorama_float.astype(np.uint8)
                
                print(f"  ✅ 圖像 {i+1} 融合成功，配準誤差: {result['registration_error']:.2f}")
            else:
                print(f"  ❌ 圖像 {i+1} 配準失敗: {result['reason']}")
        
        return panorama

# 初始化配準器
registration = ImageRegistration()
print("✅ 圖像配準器初始化完成")

### 🎯 練習任務 3.1: 實現圖像配準

**任務**: 測試圖像配準功能，並評估配準精度。

In [ ]:
def test_image_registration(img1_path, img2_path):
    """
    測試圖像配準功能
    
    Args:
        img1_path: 參考圖像路徑
        img2_path: 待配準圖像路徑
    """
    # 載入圖像
    img1 = load_image(img1_path)
    img2 = load_image(img2_path)
    
    if img1 is None or img2 is None:
        print("無法載入圖像")
        return
    
    # 調整圖像大小
    img1 = resize_image(img1, max_width=400)
    img2 = resize_image(img2, max_width=400)
    
    print(f"\n🔧 圖像配準測試")
    print(f"參考圖像: {os.path.basename(img1_path)}")
    print(f"待配準圖像: {os.path.basename(img2_path)}")
    
    # 執行配準
    start_time = time.time()
    result = registration.register_images(img1, img2)
    registration_time = (time.time() - start_time) * 1000
    
    print(f"配準處理時間: {registration_time:.1f}ms")
    
    if result['status'] == 'success':
        print(f"✅ 配準成功")
        print(f"配準誤差 (RMSE): {result['registration_error']:.2f}")
        print(f"特徵匹配點: {result['match_result']['total_matches']}")
        print(f"內點數量: {result['match_result']['inliers_count']}")
        
        # 顯示配準結果
        registered_img = result['registered_image']
        
        # 創建對比圖
        diff_img = cv2.absdiff(img1, registered_img)
        
        images = [img1, img2, registered_img, diff_img]
        titles = [
            "參考圖像",
            "原始圖像", 
            f"配準結果\nRMSE: {result['registration_error']:.2f}",
            "差異圖像"
        ]
        
        display_multiple_images(images, titles, figsize=(16, 8))
        
        # 評估配準質量
        if result['registration_error'] < 20:
            quality = "優秀"
        elif result['registration_error'] < 40:
            quality = "良好"
        elif result['registration_error'] < 60:
            quality = "一般"
        else:
            quality = "需改善"
        
        print(f"🏆 配準質量評級: {quality}")
        
    else:
        print(f"❌ 配準失敗: {result['reason']}")
        
        # 顯示原始圖像
        display_multiple_images(
            [img1, img2], 
            ["參考圖像", "待配準圖像"], 
            figsize=(10, 5)
        )

# 測試配準功能
registration_test_pairs = [
    ("../../assets/images/basic/face03.jpg", "../../assets/images/basic/faces01.jpg")
]

for img1_path, img2_path in registration_test_pairs:
    if os.path.exists(img1_path) and os.path.exists(img2_path):
        test_image_registration(img1_path, img2_path)
    else:
        print("⚠️ 測試圖像不存在，跳過配準測試")

## 挑戰4: 綜合評估系統

### 任務描述
創建一個綜合評估系統，能夠自動評估不同特徵匹配方法在各種場景下的表現。

In [ ]:
class FeatureMatchingEvaluator:
    """特徵匹配綜合評估器"""
    
    def __init__(self):
        """初始化評估器"""
        self.test_results = []
        
    def evaluate_method(self, detector_name, test_images, challenges=None):
        """
        評估特定檢測方法
        
        Args:
            detector_name: 檢測器名稱
            test_images: 測試圖像對列表
            challenges: 挑戰類型列表
        
        Returns:
            dict: 評估結果
        """
        if challenges is None:
            challenges = ['normal', 'scale', 'rotation', 'illumination']
        
        matcher = RobustFeatureMatcher(detector_name=detector_name)
        
        results = {
            'detector': detector_name,
            'challenges': {},
            'overall_score': 0,
            'processing_times': [],
            'match_counts': [],
            'success_rate': 0
        }
        
        successful_tests = 0
        total_tests = 0
        
        print(f"\n📊 評估 {detector_name} 檢測器")
        print("-" * 40)
        
        for challenge in challenges:
            challenge_results = []
            
            for img1_path, img2_path in test_images:
                if not (os.path.exists(img1_path) and os.path.exists(img2_path)):
                    continue
                
                # 載入並預處理圖像
                img1, img2 = self._load_and_preprocess(
                    img1_path, img2_path, challenge
                )
                
                if img1 is None or img2 is None:
                    continue
                
                # 執行匹配
                start_time = time.time()
                match_result = matcher.match_features(img1, img2)
                process_time = (time.time() - start_time) * 1000
                
                total_tests += 1
                
                if match_result['status'] == 'success':
                    successful_tests += 1
                    
                    # 收集性能指標
                    challenge_results.append({
                        'success': True,
                        'matches': match_result['total_matches'],
                        'inliers': match_result['inliers_count'],
                        'ratio': match_result['match_ratio'],
                        'time': process_time
                    })
                    
                    results['processing_times'].append(process_time)
                    results['match_counts'].append(match_result['total_matches'])
                    
                else:
                    challenge_results.append({
                        'success': False,
                        'time': process_time
                    })
            
            # 計算挑戰性能統計
            if challenge_results:
                success_count = sum(1 for r in challenge_results if r['success'])
                challenge_success_rate = success_count / len(challenge_results)
                
                avg_matches = np.mean([r['matches'] for r in challenge_results if r['success']]) if success_count > 0 else 0
                avg_ratio = np.mean([r['ratio'] for r in challenge_results if r['success']]) if success_count > 0 else 0
                avg_time = np.mean([r['time'] for r in challenge_results])
                
                results['challenges'][challenge] = {
                    'success_rate': challenge_success_rate,
                    'avg_matches': avg_matches,
                    'avg_ratio': avg_ratio,
                    'avg_time': avg_time
                }
                
                print(f"{challenge:12}: {challenge_success_rate:.1%} 成功率, "
                      f"{avg_matches:5.1f} 平均匹配點, {avg_time:6.1f}ms")
        
        # 計算總體得分
        results['success_rate'] = successful_tests / total_tests if total_tests > 0 else 0
        
        # 綜合評分（0-100）
        score_components = {
            'success_rate': results['success_rate'] * 40,  # 40%權重
            'match_quality': np.mean([c['avg_ratio'] for c in results['challenges'].values()]) * 30,  # 30%權重
            'speed': min(100, 1000 / np.mean(results['processing_times'])) if results['processing_times'] else 0  # 30%權重（速度倒數）
        }
        
        results['overall_score'] = sum(score_components.values())
        results['score_breakdown'] = score_components
        
        print(f"\n📈 {detector_name} 總體評估:")
        print(f"成功率: {results['success_rate']:.1%}")
        print(f"平均處理時間: {np.mean(results['processing_times']):.1f}ms" if results['processing_times'] else "N/A")
        print(f"綜合得分: {results['overall_score']:.1f}/100")
        
        self.test_results.append(results)
        return results
    
    def _load_and_preprocess(self, img1_path, img2_path, challenge):
        """
        載入並根據挑戰類型預處理圖像
        
        Args:
            img1_path, img2_path: 圖像路徑
            challenge: 挑戰類型
        
        Returns:
            tuple: 處理後的圖像對
        """
        img1 = load_image(img1_path)
        img2 = load_image(img2_path)
        
        if img1 is None or img2 is None:
            return None, None
        
        # 基本大小調整
        img1 = resize_image(img1, max_width=400)
        img2 = resize_image(img2, max_width=400)
        
        # 根據挑戰類型進行變換
        if challenge == 'scale':
            # 縮放變換
            scale_factor = 0.7
            h, w = img2.shape[:2]
            img2 = cv2.resize(img2, (int(w * scale_factor), int(h * scale_factor)))
            
        elif challenge == 'rotation':
            # 旋轉變換
            angle = 15
            h, w = img2.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            img2 = cv2.warpAffine(img2, M, (w, h))
            
        elif challenge == 'illumination':
            # 光照變換
            img2 = cv2.convertScaleAbs(img2, alpha=1.3, beta=20)
        
        return img1, img2
    
    def generate_report(self):
        """
        生成綜合評估報告
        """
        if not self.test_results:
            print("沒有測試結果可報告")
            return
        
        print("\n" + "="*60)
        print("📊 特徵匹配方法綜合評估報告")
        print("="*60)
        
        # 按總分排序
        sorted_results = sorted(self.test_results, key=lambda x: x['overall_score'], reverse=True)
        
        print("\n🏆 綜合性能排名:")
        print("-" * 50)
        
        for i, result in enumerate(sorted_results, 1):
            print(f"{i}. {result['detector']:6} - 得分: {result['overall_score']:5.1f}/100")
            print(f"   成功率: {result['success_rate']:5.1%}, "
                  f"平均時間: {np.mean(result['processing_times']):5.1f}ms" if result['processing_times'] else "N/A")
        
        # 各項指標分析
        print("\n📈 詳細性能分析:")
        print("-" * 50)
        
        for category in ['success_rate', 'speed', 'match_quality']:
            best_detector = max(sorted_results, 
                              key=lambda x: x['score_breakdown'][category])['detector']
            category_names = {
                'success_rate': '成功率',
                'speed': '處理速度',
                'match_quality': '匹配質量'
            }
            print(f"{category_names[category]:8}: {best_detector} 表現最佳")
        
        # 使用建議
        print("\n💡 使用建議:")
        print("-" * 30)
        
        best_overall = sorted_results[0]['detector']
        fastest = min(sorted_results, 
                     key=lambda x: np.mean(x['processing_times']) if x['processing_times'] else float('inf'))['detector']
        
        print(f"• 通用應用: {best_overall} (綜合表現最佳)")
        print(f"• 實時應用: {fastest} (處理速度最快)")
        print(f"• 精確匹配: SIFT (經典穩定算法)")
        print(f"• 移動設備: ORB (資源消耗低)")

# 創建評估器
evaluator = FeatureMatchingEvaluator()
print("✅ 特徵匹配評估器初始化完成")

### 🎯 練習任務 4.1: 執行綜合評估

**任務**: 對不同的特徵檢測器進行綜合評估並生成報告。

In [ ]:
# 準備測試圖像對
evaluation_test_pairs = [
    ("../../assets/images/basic/face03.jpg", "../../assets/images/basic/faces01.jpg"),
    ("../../assets/images/basic/faces.png", "../../assets/images/basic/faces01.jpg")
]

# 過濾存在的圖像對
valid_pairs = [(p1, p2) for p1, p2 in evaluation_test_pairs 
               if os.path.exists(p1) and os.path.exists(p2)]

if valid_pairs:
    print(f"🔍 找到 {len(valid_pairs)} 對有效測試圖像")
    
    # 測試不同的檢測器
    detectors_to_test = ['SIFT', 'ORB', 'BRISK', 'AKAZE']
    
    for detector in detectors_to_test:
        try:
            evaluator.evaluate_method(
                detector_name=detector,
                test_images=valid_pairs,
                challenges=['normal', 'scale', 'rotation']
            )
        except Exception as e:
            print(f"❌ {detector} 評估失敗: {e}")
    
    # 生成綜合報告
    evaluator.generate_report()
    
else:
    print("⚠️ 沒有找到有效的測試圖像對")
    print("請確保以下圖像存在:")
    for p1, p2 in evaluation_test_pairs:
        print(f"  - {p1}")
        print(f"  - {p2}")

## 挑戰總結與延伸練習

### 🎯 本練習完成的技能

1. **多特徵檢測器比較**: 掌握SIFT、ORB、BRISK、AKAZE等檢測器的使用
2. **魯棒特徵匹配**: 實現處理各種變換的穩定匹配算法
3. **圖像配準**: 基於Homography的圖像對齊技術
4. **綜合評估**: 建立完整的性能評估體系

### 📊 效能指標理解

- **特徵點數量**: 更多不一定更好，關鍵是質量
- **匹配精度**: 內點比例反映匹配的可靠性
- **處理速度**: 實時應用的關鍵指標
- **魯棒性**: 對變換的適應能力

### 🚀 進階挑戰方向

1. **自定義描述子**: 設計針對特定場景的特徵描述子
2. **深度學習特徵**: 整合深度學習特徵檢測方法
3. **實時優化**: 實現移動設備上的實時特徵匹配
4. **多視角匹配**: 處理3D重建中的多視角匹配問題

### 📚 延伸學習資源

#### 相關論文
- SIFT: "Distinctive Image Features from Scale-Invariant Keypoints" - David Lowe
- ORB: "ORB: an efficient alternative to SIFT or SURF" - Rublee et al.
- BRISK: "BRISK: Binary Robust invariant scalable keypoints" - Leutenegger et al.

#### 實際應用
- 視覺SLAM
- 3D重建
- 圖像檢索
- 擴增實境
- 文檔配準

#### 下一步學習
- 6.2.2 圖像拼接專案
- 6.2.3 影片分析任務
- 7.1 實戰專案開發

### 🏆 挑戰完成度自評

請根據完成情況勾選：

- [ ] 成功比較了至少3種不同的特徵檢測器
- [ ] 實現了魯棒的特徵匹配（處理尺度、旋轉變化）
- [ ] 完成了圖像配準並評估了精度
- [ ] 建立了綜合評估系統並生成了報告
- [ ] 理解了不同方法的優缺點和適用場景

**完成3項以上即可進入下一個中級練習！** 🎉